### Figure 1d, S1d, S1e

In [ ]:
library(maftools)
library(ComplexHeatmap)
library(dplyr)
library(circlize)
library(gridExtra)
library(ggplot2)
library(tidyr)

In [ ]:
cnv <- read.csv("./OCEC_cn.csv")
cnv_unique <- cnv[!duplicated(cnv$gene_name), ]
rownames(cnv_unique) <- cnv_unique$gene_name
cnv_unique <- cnv_unique[ , -2:-1]


In [ ]:
cnv_unique[cnv_unique>3] <- "Amplification"
cnv_unique[cnv_unique==3] <- "Gain"
cnv_unique[cnv_unique==2] <- ""
cnv_unique[cnv_unique==1] <- "Heterozygous Loss"
cnv_unique[cnv_unique==0] <- "Homozygous Deletion"

key_genes_cnv <- cnv_unique[c(
                            "MYC","PIK3CA","E2F1","RHEB","AKT3","PIK3CB","RPTOR","CCND1","CCND2","EGFR","BRAF","CDK2","E2F3","PIK3R2","KRAS","CDK4",
                            "MDM4","PTEN","AKT1","TSC2","ERBB2","CCND3","AKT2","SOS1","MYCN","RB1","AXIN1","INPP4B","ATM","CCNE1","TP53","CDKN2A","CDKN2B",
                            "RAC1","CHEK2","ZNRF3","APC","PIK3R1","NF1","STK11","MGA"
                        ),]

cn_matrix <- as.matrix(key_genes_cnv)

In [ ]:
clinical_data <- read.csv("./CN_clinical_df.csv",row.names=1)
clinical_data$TP53 <- as.character(clinical_data$TP53)
clinical_data$PTEN <- as.character(clinical_data$PTEN)
clinical_data$ARID1A <- as.character(clinical_data$ARID1A)
clinical_data$PIK3CA <- as.character(clinical_data$PIK3CA)
clinical_data$RNA_cluster <- as.character(clinical_data$RNA_cluster)
clinical_data$Tumor_Type <- factor(clinical_data$Tumor_Type, level=c("EC","OC"))

clinical_data <- clinical_data %>%
  mutate(RNA_cluster = case_when(
    RNA_cluster == "1" ~ "Immunoreative",
    RNA_cluster == "2" ~ "Proliferating",
    RNA_cluster == "3" ~ "Mesenchymal",
    RNA_cluster == "4" ~ "Stem-like",
    TRUE ~ as.character(RNA_cluster)
  ))

clinical_data[is.na(clinical_data)] <- "Unknown"
clinical_data[clinical_data==""] <- "Unknown"
clinical_data <- clinical_data[order(clinical_data$Tumor_Type), ]
filter_cn_matrix <- cn_matrix[,rownames(clinical_data)]

In [ ]:
EC_cn_matrix <- filter_cn_matrix[,1:127]
OC_cn_matrix <- filter_cn_matrix[,128:ncol(filter_cn_matrix)]


cn_types <- c("Amplification","Gain","Heterozygous Loss","Homozygous Deletion")
cn_col_list <- c("#de4431","#f2aa70","#a7b9d2","#4580b8")
cn_colors <- setNames(cn_col_list, cn_types)

cn_alter_fun_list <- list()
for (cn_type in cn_types) {
  cn_alter_fun_list[[cn_type]] <- local({
    cn_type_copy <- cn_type
    function(x, y, w, h) {
      grid.rect(x, y, w, h, gp = gpar(fill = cn_colors[cn_type_copy], col = NA))
    }
  })
}

cn_alter_fun_list$background <- function(x, y, w, h) {
  grid.rect(x, y, w, h, gp = gpar(fill = "#ebebeb", col = NA))
}

cn_priority <- setNames(seq_along(cn_types), cn_types)

In [ ]:
EC_num_matrix <- apply(EC_cn_matrix, 2, function(x) {
  sapply(x, function(y) ifelse(y %in% names(cn_priority), cn_priority[y], 0))
})
EC_num_matrix[EC_num_matrix==0] <- 5
EC_sample_order <- do.call(order, as.data.frame(t(EC_num_matrix)))

OC_num_matrix <- apply(OC_cn_matrix, 2, function(x) {
  sapply(x, function(y) ifelse(y %in% names(cn_priority), cn_priority[y], 0))
})
OC_num_matrix[OC_num_matrix==0] <- 5
OC_sample_order <- do.call(order, as.data.frame(t(OC_num_matrix)))

OC_sample_order <- OC_sample_order+127
sample_order <- c(EC_sample_order,OC_sample_order)

order_matrix <- filter_cn_matrix[,sample_order]
clinical_data <- clinical_data[colnames(order_matrix), ]

In [ ]:
ha <- HeatmapAnnotation(
  Tumor_Type = clinical_data$Tumor_Type,
  Subtype = clinical_data$Tumor_SubType,
  Age = clinical_data$Age_state,
  Recurrence = clinical_data$Recurrence_Status,
  Purity = clinical_data$purity_state,
  Tumor_Grade = clinical_data$Tumor_Grade,
  Tumor_Stage = clinical_data$Tumor_Stage,
  MSI = clinical_data$MSI_state,
  RNA_cluster = clinical_data$RNA_cluster,
  TP53 = clinical_data$TP53,
  PTEN = clinical_data$PTEN,
  ARID1A = clinical_data$ARID1A,
  PIK3CA = clinical_data$PIK3CA,
  annotation_height = unit(8, "mm"),
  show_annotation_name = TRUE,
  annotation_name_side = "left",
  col = list(
    Tumor_Type = c("OC" = "#ab526a", "EC" = "#d68081", 'Unknown' = "#dcdcdc"),
    Subtype = c("Mucinous" = "#b1bcf4", "Serous" = "#65a56d", "Clear cell" = "#f4e88d", "Endometrial" = "#eab768", "Sarcoma" = "#cf5b5d",'Others' = "#dcdcdc"),
    Age = c("High" = "#d87559", "Medium" = "#e6a76c", "Low" = "#e3c577", 'Unknown' = "#dcdcdc"),
    Recurrence = c("1" = "#785190", "0" = "#6f87bb", 'Unknown' = "#dcdcdc"),
    Purity = c("High" = "#345b7c", "Medium" = "#5896aa", "Low" = "#91cfe3", 'Unknown' = "#dcdcdc"),
    Tumor_Grade = c("1" = "#a1a5bf", "2" = "#656e96", "3" = "#475482", 'Unknown' = "#dcdcdc"),
    Tumor_Stage = c("I" = "#a6c692", "II" = "#92b17f", "III" = "#7e9d6b", "IV" = "#6b8959", 'Unknown' = "#dcdcdc"),
    MSI = c("MSI-low" = "#62b58f", "MSI-high" = "#e7cc5e", 'Unknown' = "#dcdcdc"),
    RNA_cluster = c("Immunoreative"="#b1bcf4", "Proliferating"="#65a56d", "Mesenchymal"="#eab768", "Stem-like"="#f4e88d", 'Unknown' = "#dcdcdc"),
    TP53 = c("1" = "#393939", "0" = "#e4e4e4", 'Unknown' = "#dcdcdc"),
    PTEN = c("1" = "#393939", "0" = "#e4e4e4", 'Unknown' = "#dcdcdc"),
    ARID1A = c("1" = "#393939", "0" = "#e4e4e4", 'Unknown' = "#dcdcdc"),
    PIK3CA = c("1" = "#393939", "0" = "#e4e4e4", 'Unknown' = "#dcdcdc")
  ),
  annotation_legend_param = list(
    Tumor_Type = list(border = TRUE, border_gp = gpar(col = "white")),
    Subtype = list(border = TRUE, border_gp = gpar(col = "white")),
    Age = list(border = TRUE, border_gp = gpar(col = "white")),
    Recurrence = list(border = TRUE, border_gp = gpar(col = "white")),
    Purity = list(border = TRUE, border_gp = gpar(col = "white")),
    Tumor_Grade = list(border = TRUE, border_gp = gpar(col = "white")),
    Tumor_Stage = list(border = TRUE, border_gp = gpar(col = "white")),
    MSI = list(border = TRUE, border_gp = gpar(col = "white")),
    RNA_cluster = list(border = TRUE, border_gp = gpar(col = "white")),
    TP53 = list(border = TRUE, border_gp = gpar(col = "white")),
    PTEN = list(border = TRUE, border_gp = gpar(col = "white")),
    ARID1A = list(border = TRUE, border_gp = gpar(col = "white")),
    PIK3CA = list(border = TRUE, border_gp = gpar(col = "white"))
  )
)

In [ ]:
mut_onco <- oncoPrint(order_matrix,
          alter_fun = cn_alter_fun_list,
          col = cn_colors,
          top_annotation = ha,
          remove_empty_columns = FALSE,
          remove_empty_rows = FALSE,
          column_order = 1:ncol(filter_cn_matrix),
        #   column_title = "Samples",
          column_title = NULL,
          right_annotation = NULL,
          left_annotation = NULL,
          # row_title = "Genes",
          show_column_names = FALSE,
          row_names_side = "right",
          show_row_names = TRUE)

print(mut_onco)